# Build Experiment Dataset

## Imports

In [ ]:
import io 
import itertools

import numpy as np
import pandas as pd

import altair as alt

import panel as pn

## Setup

In [ ]:
pn.extension("tabulator", "vega", notifications=True, console_output="disable")
template = pn.template.BootstrapTemplate(title="OIV DataIn Generator")

In [ ]:
alt.data_transformers.disable_max_rows()

## UI

### Functions

In [ ]:
def expand_genotypes(genotypes, counts):
    nested_list = [[g for _ in range(q)] for g, q in zip(genotypes, counts)]
    return nested_list


def flatten_genotypes(nested_list):
    flatten_list = []
    for l in nested_list:
        for e in l:
            flatten_list.append(e)
    flatten_list = np.array(flatten_list)
    np.random.shuffle(flatten_list)
    flatten_list

    out = []
    for fd in flatten_list:
        for _ in range(3):
            out.append(fd)
    return out


def build_table(genptypes, strains, first_plate_number):
    plates_count = (len(genptypes) // 12) + 1

    row = ["a", "b", "c"]
    column = [1, 2, 3, 4]

    return pd.concat(
        [
            pd.DataFrame(
                data=itertools.product(
                    [
                        first_plate_number + i + (idx * plates_count)
                        for i in range(plates_count)
                    ],
                    column,
                    row,
                ),
                columns=["plaque", "colonne", "ligne"],
            )
            .assign(
                genotype=genptypes
                + [np.nan for _ in range(plates_count * 12 - len(genptypes))]
            )
            .assign(isolat=current_strain)
            for idx, current_strain in enumerate(strains)
        ]
    ).reset_index(drop=True)

### Widgets

In [ ]:
csv_input = pn.widgets.FileInput(accept=".csv", sizing_mode="scale_width")
csv_out = pn.widgets.Tabulator(
    pagination="local", page_size=12, header_filters=True, sizing_mode="scale_width"
)
ti_strains = pn.widgets.TextAreaInput(
    name="Isolats",
    placeholder="Entrer les isolats séparés par des ';'",
    value="Colmar;",
    sizing_mode="stretch_width",
)
bt_generate = pn.widgets.Button(name="Générer", sizing_mode="scale_width")

filename, bt_download = csv_out.download_menu(
    text_kwargs={'name': 'Enter filename', 'value': 'experiment.csv'},
    button_kwargs={'name': 'Download table'}
)

vg_output = pn.pane.Vega()

### Callbacks

In [ ]:
def build_csv(event):
    try:
        csv = pd.read_csv(io.BytesIO(csv_input.value), sep=";")
    except:
        pn.state.notifications.error("Please load a valid file", duration=5000)
        return

    csv.columns = map(str.lower, csv.columns)
    csv = csv.rename(columns={"gen": "genotype"})
    csv_ctrl = csv[csv.ctrl == 1].drop(["ctrl", "cpm", "id"], axis=1)
    csv_exp = csv[csv.ctrl == 0].drop(["ctrl", "cpm", "id"], axis=1)

    isolats = [s for s in ti_strains.value.replace("\n", ";").split(";") if s]
    try:
        assert len(isolats) > 0
    except:
        pn.state.notifications.error(
            "At least one strain must be present", duration=5000
        )
        return
    ctrl = build_table(
        flatten_genotypes(expand_genotypes(csv_ctrl.genotype, csv_ctrl.nb_col)),
        isolats,
        first_plate_number=1,
    )
    exp = build_table(
        flatten_genotypes(expand_genotypes(csv_exp.genotype, csv_exp.nb_col)),
        isolats,
        first_plate_number=ctrl.plaque.max() + 1,
    )
    out = (
        pd.concat([ctrl, exp])
        .merge(csv.drop(["nb_col"], axis=1), on="genotype", how="left")
        .assign(x=lambda x: x.colonne)
        .assign(y=lambda x: np.where(x.ligne == "a", 1, np.where(x.ligne == "b", 2, 3)))
        .assign(ctrl=lambda x: x.ctrl.fillna("vide"))
    )

    csv_out.value = out

    csv_out.editors = {
        "ligne": {"type": "list", "valuesLookup": True},
        "genotype": {"type": "list", "valuesLookup": True},
        "isolat": {"type": "list", "valuesLookup": True},
        "id": {"type": "list", "valuesLookup": True},
        "cpm": {"type": "list", "valuesLookup": True},
    }

    vg_output.object = alt.hconcat(
        *[
            alt.Chart(out.fillna("vide"))
            .mark_bar()
            .encode(
                x=alt.X(c, axis=alt.Axis(labelAngle=-45)),
                y="count()",
                color="ctrl",
                tooltip=["genotype", "count()"],
            ).properties(height=250)
            for c in ["genotype", "isolat"]
        ]
    ).interactive()


bt_generate.on_click(build_csv)

### Show

In [ ]:
cd_settings = pn.Card(
    csv_input,
    ti_strains,
    bt_generate,
    title="Experiment setup",
    sizing_mode="scale_width",
)

cd_download = pn.Card(
    filename,
    bt_download,
    title="Download table",
    sizing_mode="scale_width",
)

template.sidebar.append(cd_settings)
template.sidebar.append(cd_download)
template.main.append(pn.Column(csv_out, vg_output))
template.servable()